In [1]:
import numpy as np
import pandas as pd

In [4]:
# Load the data
recipes = pd.read_csv("dataset/recipes_df.csv")
recipes.head()

,cooking_method,ingredients,recipe_name,tags,Vegetarian&Desserts,Others&D,Vegetarian,Others,Dairy Free,Gluten Free,Low Carb,Low Fat,Low Sodium
0,['Set the racks in the middle and upper thirds...,"['1 tablespoons extra virgin olive oil', '1 cu...",Mediterranean Sea Bass,"Dairy Free, Gluten Free, Keto Recipes, Kid Fri...",0,1,0,1,1,1,1,0,0
1,['Place the eggs in the air fryer basket and c...,"['4 large eggs', 'Salt (black pepper, everythi...",Air Fryer Hard Boiled Eggs,"Air Fryer Recipes, Dairy Free, Gluten Free, Ke...",1,0,1,0,1,1,1,0,0
2,"['Air Fryer directions:', 'Preheat air fryer t...","['olive oil spray', '4 about 5 ounce each salm...",Air Fryer Basil-Parmesan Salmon,"Air Fryer Recipes, Gluten Free, Keto Recipes, ...",0,1,0,1,0,1,1,0,0
3,"['Preheat the oven to 400F.', 'Pour 2 tablespo...",['1/2 cup freshly grated Parmesan (not pre-gra...,Everything Parmesan Crisps,"Gluten Free, Keto Recipes, Kid Friendly, Low C...",1,0,1,0,0,1,1,0,0
4,['Cook potatoes in a large pot of salted water...,['3 1/2 pounds new potatoes (about 10 peeled a...,Potato and Green Bean Salad,"Dairy Free, Gluten Free, Kid Friendly, Vegetar...",1,0,1,0,1,1,0,0,0


In [11]:
ingredients = pd.read_csv("dataset/ingredients_v.csv")
ingredients.head()

,tellini
0,brewed black tea
1,baby spring greens
2,rouette cookies
3,Sauza Silver
4,gnish zucchini


In [14]:
ingredients.tellini.unique

<bound method Series.unique of 0                 brewed black tea
1               baby spring greens
2                  rouette cookies
3                     Sauza Silver
4                   gnish zucchini
                   ...            
82056                Milano salami
82057                   pistachios
82058         pork loin medallions
82059                cloves bottle
82060    coho salmon fillets bones
Name: tellini, Length: 82061, dtype: object>

## Sono troppi, quindi facciamo su meno ricette, estraendo di nuovo gli ingredienti da tipo 5k ricette.

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch

c:\Users\Giamm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
df=pd.read_csv("dataset/recipes_df.csv")

In [21]:
df_veg=df[df.Vegetarian==True]
df_notveg=df[df.Vegetarian==False]

In [24]:
df_veg=df_veg.sample(n=2500)
df_notveg=df_notveg.sample(n=2500)

In [26]:
df=pd.concat([df_veg,df_notveg])

In [30]:
#shuffle
df = df.sample(frac = 1)

In [39]:
#save to csv
df.to_csv('sbunecksets/recipes_df.csv', index=False)

In [32]:
def get_ingredients(recipe: str, ner_result: list):
    ingredients = []
    last_added = 0
    b_word = 'B-FOOD'
    i_word = 'I-FOOD'

    for i in range(len(ner_result)):

        if ner_result[i]['entity'] == b_word:
            # check if previous word was a segmentation of the same one
            if ner_result[i]['word'].startswith('#') and ner_result[i-1]['entity'] == b_word:
                # if (for any reason (it happens)) the first word is a segment, we ignore it
                if last_added == 0:
                    continue
                ingredients[last_added-1] += recipe[ner_result[i]['start'] : ner_result[i]['end']]
            else:
                # get the ingredient from the recipe given its position
                ingredients.append(recipe[ner_result[i]['start'] : ner_result[i]['end']])
                last_added += 1

        elif ner_result[i]['entity'] == i_word:
            # check if segmentation is occurring
            if ner_result[i]['word'].startswith('#'):
                # if (for any reason (it happens)) the first word is a segment, we ignore it
                if last_added == 0:
                    continue
                ingredients[last_added-1] += recipe[ner_result[i]['start'] : ner_result[i]['end']]
            elif last_added == 0:
                ingredients.append(recipe[ner_result[i]['start'] : ner_result[i]['end']])
                last_added += 1
            else:
                ingredients[last_added-1] = ingredients[last_added-1] + ' ' + recipe[ner_result[i]['start'] : ner_result[i]['end']]

    return ingredients

In [33]:
ingredients_v = set()

tokenizer = AutoTokenizer.from_pretrained("Dizex/FoodBaseBERT")
model = AutoModelForTokenClassification.from_pretrained("Dizex/FoodBaseBERT")

if torch.cuda.is_available():
    model = model.to('cuda')
    print("Model moved to GPU.")
else:
    print("CUDA is not available. Model will run on CPU.")

pipe = pipeline("ner", model=model, tokenizer=tokenizer)

for i, recipe in tqdm(enumerate(df.ingredients), total=len(df.ingredients)):
    ner_result = pipe(recipe)
    ingredients = get_ingredients(recipe, ner_result)
    for ingredient in ingredients:
        ingredients_v.add(ingredient)

CUDA is not available. Model will run on CPU.


100%|██████████| 5000/5000 [22:54<00:00,  3.64it/s]


In [37]:
import csv

PATH = 'sbunecksets/ingredients_v.csv'

with open(PATH, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    for ingredient in ingredients_v:
        writer.writerow([ingredient])

print(f"Set saved to '{PATH}'.")

Set saved to 'sbunecksets/ingredients_v.csv'.


# RUNNARE DA QUI IN POI CAMBIANDO I PATH A QUESTI TRE df (sono tutti con le 5000 ricette)

In [151]:
df=pd.read_csv('sbunecksets/ingredients_list_5k.csv')
df2=pd.read_csv('sbunecksets/recipes_df.csv')

In [152]:
df=pd.concat([df, df2["Vegetarian"]], axis=1)

In [153]:
df_vocabulary=pd.read_csv('sbunecksets/ingredients_v.csv')

# Ingredients_list_5k E' ANCORA SPORCO 

In [154]:
s=df["ingredients"][0]
s

"['olive oil', 'fresh salmon', 'scallops', 'calamari rings', 'large shrimp', 'Old', 'Bay seasoning', 'd', 'red bell peppers', 'd', 'green bell peppers', 'yellow bell peppers', 'd tomato', 'd onion', 'sliced mushrooms', 'sliced zucchini', 'light soy sauce', 'white wine']"

In [155]:
s=s.replace('[', '').replace(']','').replace('\n', '').replace('\'' , '')

In [156]:
s=np.array(s.split(', '))
s

array(['olive oil', 'fresh salmon', 'scallops', 'calamari rings',
       'large shrimp', 'Old', 'Bay seasoning', 'd', 'red bell peppers',
       'd', 'green bell peppers', 'yellow bell peppers', 'd tomato',
       'd onion', 'sliced mushrooms', 'sliced zucchini',
       'light soy sauce', 'white wine'], dtype='<U19')

In [157]:
s[1]

'fresh salmon'

In [158]:
i=0
for s in df["ingredients"]:
    s=s.replace('[', '').replace(']','').replace('\n', '').replace('\'' , '')
    df["ingredients"][i]=np.array(s.split(', '))
    i=i+1

C:\Users\Giamm\AppData\Local\Temp\ipykernel_16484\2293704621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i]=np.array(s.split(', '))
C:\Users\Giamm\AppData\Local\Temp\ipykernel_16484\2293704621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i]=np.array(s.split(', '))
C:\Users\Giamm\AppData\Local\Temp\ipykernel_16484\2293704621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingred

In [159]:
df.head()

,ingredients,Vegetarian
0,"[olive oil, fresh salmon, scallops, calamari r...",0
1,"[beef tenderloin fillet, Salt, pepper, butter,...",0
2,"[eggplants, kosher salt, extra, -, virgin oliv...",1
3,"[Becha, mel Sauce, white wine, ground mustard,...",1
4,"[quality beef, Charolais beef, Burgundian wine...",0


# One Hot Encoding

In [45]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder()


In [50]:

# 2. FIT
enc.fit(df_vocabulary)

# 3. Transform
onehotlabels = enc.transform(df_vocabulary).toarray()
onehotlabels.shape

(13403, 13403)

# PROCODIO

problemi col parsing come al solito

In [160]:
i=0
s=df["ingredients"][0]
a=np.zeros([1,13403])
for s in df["ingredients"]:
    for g in s:
        g=enc.transform([[g]])
        a=a+g
    a=a>0
    df["ingredients"][i]=a
    i=i+1

c:\Users\Giamm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\Giamm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\Giamm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\Giamm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\Giamm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder wa

ValueError: Found unknown categories ['"Confectioners  sugar"'] in column 0 during transform

# Testiamo LogReg

In [ ]:
X=df["ingredients"]
y=df["Vegetarian"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [ ]:
scaler = StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
# fitting the model
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)
LogisticRegression()
# predictions
y_pred = logreg.predict(X_test_scaled)
y_pred_train = logreg.predict(X_train_scaled)

In [ ]:
# metrics
results_logreg = evaluate_model(logreg, X_train_scaled, y_train, X_test_scaled, y_test)